برای اجرای نوت‌بوک‌های زیر، اگر هنوز انجام نداده‌اید، باید مدلی را که از `text-embedding-ada-002` به عنوان مدل پایه استفاده می‌کند، مستقر کنید و نام استقرار را در فایل .env به عنوان `AZURE_OPENAI_EMBEDDINGS_ENDPOINT` تنظیم کنید.


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

در مرحله بعد، قصد داریم شاخص جاسازی را در یک دیتافریم پاندا بارگذاری کنیم. شاخص جاسازی در یک فایل JSON به نام `embedding_index_3m.json` ذخیره شده است. شاخص جاسازی شامل جاسازی‌ها برای هر یک از متن‌های یوتیوب تا اواخر اکتبر ۲۰۲۳ می‌باشد.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

در مرحله بعد، ما قصد داریم تابعی به نام `get_videos` ایجاد کنیم که در شاخص جاسازی (Embedding Index) به دنبال پرس‌وجو جستجو می‌کند. این تابع ۵ ویدیوی برتر که بیشترین شباهت را به پرس‌وجو دارند، بازمی‌گرداند. عملکرد تابع به شرح زیر است:

1. ابتدا یک نسخه کپی از شاخص جاسازی ایجاد می‌شود.
2. سپس، جاسازی پرس‌وجو با استفاده از API جاسازی OpenAI محاسبه می‌شود.
3. سپس یک ستون جدید به نام `similarity` در شاخص جاسازی ایجاد می‌شود. ستون `similarity` شامل شباهت کسینوسی بین جاسازی پرس‌وجو و جاسازی هر بخش ویدیو است.
4. سپس شاخص جاسازی بر اساس ستون `similarity` فیلتر می‌شود. شاخص جاسازی به گونه‌ای فیلتر می‌شود که فقط ویدیوهایی که شباهت کسینوسی آنها برابر یا بیشتر از ۰.۷۵ است، شامل شوند.
5. در نهایت، شاخص جاسازی بر اساس ستون `similarity` مرتب شده و ۵ ویدیوی برتر بازگردانده می‌شوند.


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

این تابع بسیار ساده است، فقط نتایج جستجو را چاپ می‌کند.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. ابتدا، شاخص جاسازی شده در یک دیتافریم Pandas بارگذاری می‌شود.
2. سپس از کاربر خواسته می‌شود تا یک پرس‌وجو وارد کند.
3. سپس تابع `get_videos` برای جستجو در شاخص جاسازی شده بر اساس پرس‌وجو فراخوانی می‌شود.
4. در نهایت، تابع `display_results` برای نمایش نتایج به کاربر فراخوانی می‌شود.
5. سپس از کاربر خواسته می‌شود تا پرس‌وجوی دیگری وارد کند. این روند تا زمانی که کاربر `exit` را وارد کند ادامه می‌یابد.

![](../../../../translated_images/fa/notebook-search.1e320b9c7fcbb0bc.webp)

از شما خواسته می‌شود تا یک پرس‌وجو وارد کنید. یک پرس‌وجو وارد کرده و کلید اینتر را فشار دهید. برنامه فهرستی از ویدیوهای مرتبط با پرس‌وجو را بازمی‌گرداند. همچنین برنامه لینکی به محل پاسخ سوال در ویدیو ارائه می‌دهد.

در اینجا چند پرس‌وجو برای امتحان کردن آمده است:

- Azure Machine Learning چیست؟
- شبکه‌های عصبی کانولوشنی چگونه کار می‌کنند؟
- شبکه عصبی چیست؟
- آیا می‌توانم از Jupyter Notebooks با Azure Machine Learning استفاده کنم؟
- ONNX چیست؟


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما در تلاش برای دقت هستیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است حاوی خطاها یا نواقصی باشند. سند اصلی به زبان بومی خود باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حیاتی، ترجمه حرفه‌ای انسانی توصیه می‌شود. ما مسئول هیچ گونه سوءتفاهم یا تفسیر نادرستی که از استفاده این ترجمه ناشی شود، نیستیم.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
